In [12]:
#!pip install -r "C:\Users\ilang\Downloads\requirements.txt"

In [15]:
#pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
#pip install ftfy regex tqdm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
os.system("git --version")

0

In [3]:
#pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to c:\users\ilang\appdata\local\temp\pip-req-build-xav1ktkm
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369633 sha256=6d6658920366614409475f3c063b5383a7eda4289eb88b0ecfd5b0b1f8aa3ef7
  Stored in directory: C:\Users\ilang\AppData\Local\Temp\pip-ephem-wheel-cache-vsqdj3xu\wheels\35\3e\df\3d24cbfb3b6a06f17a2bfd7d1138900d4365d9028aa8f6e92f
Successfully built clip
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git 'C:\Users\ilang\AppData\Local\Temp\pip-req-build-xav1ktkm'

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
#pip install faiss-cpu


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import boto3
import tabula
import faiss
import json
import base64
import pymupdf
import requests
import os
import logging
import numpy as np
import warnings
from tqdm import tqdm
from botocore.exceptions import ClientError
from langchain_text_splitters import RecursiveCharacterTextSplitter
from IPython import display


logger = logging.getLogger(__name__)
logger.setLevel(logging.ERROR)

warnings.filterwarnings("ignore")

In [17]:
# Downloading the dataset - URL of the "Attention Is All You Need" paper (Replace it with the URL of the PDF file/dataset you want to download)
url = "https://arxiv.org/pdf/1706.03762.pdf"

# Set the filename and filepath
filename = "attention_paper.pdf"
filepath = os.path.join("data", filename)

# Create the data directory if it doesn't exist
os.makedirs("data", exist_ok=True)

# Download the file
response = requests.get(url)
if response.status_code == 200:
    with open(filepath, 'wb') as file:
        file.write(response.content)
    print(f"File downloaded successfully: {filepath}")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")
print(filepath)


File downloaded successfully: data\attention_paper.pdf
data\attention_paper.pdf


In [19]:
# Display the PDF file
import os
from IPython.display import IFrame
filepath = "..\\data\\attention_paper.pdf"
display.IFrame(filepath, width=1000, height=600)

In [23]:
import os
print(os.getcwd())

C:\Users\ilang\data


In [25]:
# Create the directories
def create_directories(base_dir):
    directories = ["images", "text", "tables", "page_images"]
    for dir in directories:
        os.makedirs(os.path.join(base_dir, dir), exist_ok=True)

# Process tables
def process_tables(doc, page_num, base_dir, items):
    try:
        tables = tabula.read_pdf(filepath, pages=page_num + 1, multiple_tables=True)
        if not tables:
            return
        for table_idx, table in enumerate(tables):
            table_text = "\n".join([" | ".join(map(str, row)) for row in table.values])
            table_file_name = f"{base_dir}/tables/{os.path.basename(filepath)}_table_{page_num}_{table_idx}.txt"
            with open(table_file_name, 'w') as f:
                f.write(table_text)
            items.append({"page": page_num, "type": "table", "text": table_text, "path": table_file_name})
    except Exception as e:
        print(f"Error extracting tables from page {page_num}: {str(e)}")

# Process text chunks
def process_text_chunks(text, text_splitter, page_num, base_dir, items):
    chunks = text_splitter.split_text(text)
    for i, chunk in enumerate(chunks):
        text_file_name = f"{base_dir}/text/{os.path.basename(filepath)}_text_{page_num}_{i}.txt"
        with open(text_file_name, 'w', encoding='utf-8') as f:
            f.write(chunk)
        items.append({"page": page_num, "type": "text", "text": chunk, "path": text_file_name})

# Process images
def process_images(page, page_num, base_dir, items):
    images = page.get_images()
    for idx, image in enumerate(images):
        xref = image[0]
        pix = pymupdf.Pixmap(doc, xref)
        image_name = f"{base_dir}/images/{os.path.basename(filepath)}_image_{page_num}_{idx}_{xref}.png"
        pix.save(image_name)
        with open(image_name, 'rb') as f:
            encoded_image = base64.b64encode(f.read()).decode('utf8')
        items.append({"page": page_num, "type": "image", "path": image_name, "image": encoded_image})

# Process page images
def process_page_images(page, page_num, base_dir, items):
    pix = page.get_pixmap()
    page_path = os.path.join(base_dir, f"page_images/page_{page_num:03d}.png")
    pix.save(page_path)
    with open(page_path, 'rb') as f:
        page_image = base64.b64encode(f.read()).decode('utf8')
    items.append({"page": page_num, "type": "page", "path": page_path, "image": page_image})

In [26]:
import fitz  # PyMuPDF
from tqdm import tqdm
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
doc = pymupdf.open(filepath)
num_pages = len(doc)
base_dir = "data"

# Creating the directories
create_directories(base_dir)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=200, length_function=len)
items = []

# Process each page of the PDF
for page_num in tqdm(range(num_pages), desc="Processing PDF pages"):
    page = doc[page_num]
    text = page.get_text()
    process_tables(doc, page_num, base_dir, items)
    process_text_chunks(text, text_splitter, page_num, base_dir, items)
    process_images(page, page_num, base_dir, items)
    process_page_images(page, page_num, base_dir, items)

Processing PDF pages:   0%|                                                                                  | 0/15 [00:00<?, ?it/s]Failed to import jpype dependencies. Fallback to subprocess.
No module named 'jpype'
Processing PDF pages:  20%|██████████████▊                                                           | 3/15 [00:04<00:20,  1.69s/it]Got stderr: Sept 24, 2025 9:01:15 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode

Processing PDF pages:  40%|█████████████████████████████▌                                            | 6/15 [00:08<00:12,  1.42s/it]

Error extracting tables from page 5: 'utf-8' codec can't decode byte 0xb7 in position 1027: invalid start byte


Processing PDF pages:  53%|███████████████████████████████████████▍                                  | 8/15 [00:11<00:09,  1.34s/it]

Error extracting tables from page 7: 'utf-8' codec can't decode byte 0xb7 in position 1440: invalid start byte


Processing PDF pages:  60%|████████████████████████████████████████████▍                             | 9/15 [00:12<00:07,  1.33s/it]

Error extracting tables from page 8: 'utf-8' codec can't decode byte 0xd7 in position 2962: invalid continuation byte


Processing PDF pages: 100%|█████████████████████████████████████████████████████████████████████████| 15/15 [00:22<00:00,  1.53s/it]


In [27]:
import fitz

In [28]:
[i for i in items if i['type'] == 'text'][0]

{'page': 0,
 'type': 'text',
 'text': 'Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or',
 'path': 'data/text/attention_paper.pdf_text_0_0.txt'}

In [29]:
[i for i in items if i['type'] == 'table'][0]

{'page': 9,
 'type': 'table',
 'text': 'Vinyals & Kaiser el al. (2014) [37] | WSJ only, discriminative | 88.3\nPetrov et al. (2006) [29] | WSJ only, discriminative | 90.4\nZhu et al. (2013) [40] | WSJ only, discriminative | 90.4\nDyer et al. (2016) [8] | WSJ only, discriminative | 91.7\nTransformer (4 layers) | WSJ only, discriminative | 91.3\nZhu et al. (2013) [40] | semi-supervised | 91.3\nHuang & Harper (2009) [14] | semi-supervised | 91.3\nMcClosky et al. (2006) [26] | semi-supervised | 92.1\nVinyals & Kaiser el al. (2014) [37] | semi-supervised | 92.1\nTransformer (4 layers) | semi-supervised | 92.7\nLuong et al. (2015) [23] | multi-task | 93.0\nDyer et al. (2016) [8] | generative | 93.3',
 'path': 'data/tables/attention_paper.pdf_table_9_0.txt'}

In [6]:
import clip
import torch

# Check if a GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the CLIP model
model, preprocess = clip.load("ViT-B/32", device=device)

print("CLIP model loaded successfully!")

CLIP model loaded successfully!


In [17]:
import clip
import torch
from PIL import Image

# Load the model and preprocessing pipeline
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# Define inputs
text_inputs = ["a golden retriever"]
image_path = "dog2.jpg"

# Preprocess the text
text_tokens = clip.tokenize(text_inputs).to(device)

# Preprocess the image
image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)

# Generate embeddings
with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text_tokens)

# Normalize embeddings
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)

# Compute similarity scores
similarity = (image_features @ text_features.T).cpu().numpy()

print("Similarity scores between the image and text queries:")
for i, text in enumerate(text_inputs):
    print(f"{text}: {similarity[0][i]:.4f}")

Similarity scores between the image and text queries:
a golden retriever: 0.3222


In [2]:
import faiss
import numpy as np

# Create a random dataset of 100,000 vectors (128 dimensions)
d = 128  # Dimension of vectors
nb = 100000  # Number of vectors
np.random.seed(1234)
data = np.random.random((nb, d)).astype('float32')

# Build a FAISS index (Flat index for exact search)
index = faiss.IndexFlatL2(d)  # L2 distance
index.add(data)  # Add vectors to the index

# Query the index
query = np.random.random((1, d)).astype('float32')  # Single query vector
k = 5  # Number of nearest neighbors to retrieve
distances, indices = index.search(query, k)

print(f"Top {k} nearest neighbors: {indices}")

Top 5 nearest neighbors: [[13262 17620 36245 39869 60409]]


In [1]:
nearest_vectors = data[indices[0]] 
nearest_vectors

NameError: name 'data' is not defined

In [3]:
#pip install transformers torch torchvision Pillow

   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
    --------------------------------------- 0.3/11.6 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.6 MB 1.0 MB/s eta 0:00:11
   --- ------------------------------------ 1.0/11.6 MB 1.6 MB/s eta 0:00:07
   ------ --------------------------------- 1.8/11.6 MB 2.1 MB/s eta 0:00:05
   ---------- ----------------------------- 3.1/11.6 MB 3.0 MB/s eta 0:00:03
   ------------------- -------------------- 5.8/11.6 MB 4.6 MB/s eta 0:00:02
   -------------------------- ------------- 7.6/11.6 MB 5.3 MB/s eta 0:00:01
   -------------------------- ------------- 7.6/11.6 MB 5.3 MB/s eta 0:00:01
   ----------------------------- ---------- 8.7/11.6 MB 4.6 MB/s eta 0:00:01
   -------------------------------


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [34]:
text_dir = "../data/data/text"
table_dir = "../data/data/tables"
image_dir = "../data/data/images"

In [35]:
import os
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
from tqdm import tqdm
import numpy as np
import faiss


def generate_multimodal_embeddings(prompt=None, image_path=None, output_embedding_length=384):
    """
    Generate multimodal embeddings using OpenAI's CLIP model.

    Args:
        prompt (str): The text prompt to encode.
        image_path (str): The path to the image file to encode.
        output_embedding_length (int): The desired dimension of output embeddings.
    Returns:
        dict: A dictionary containing text and/or image embeddings.
    """
    if not prompt and not image_path:
        raise ValueError("Please provide either a text prompt, an image path, or both as input")
    
    # Load the CLIP model and processor
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

    # Initialize inputs
    inputs = {}
    if prompt:
        inputs["text"] = prompt.strip()
    if image_path:
        try:
            image = Image.open(image_path).convert("RGB")
            inputs["images"] = image
        except Exception as e:
            print(f"Error loading image: {image_path} - {e}")
            return None

    # Process inputs for the model
    inputs = processor(**inputs, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        text_embeddings = None
        image_embeddings = None

        if "text" in inputs:
            text_embeddings = model.get_text_features(input_ids=inputs["input_ids"])
            text_embeddings = text_embeddings / text_embeddings.norm(dim=-1, keepdim=True)
        if "images" in inputs:
            image_embeddings = model.get_image_features(pixel_values=inputs["pixel_values"])
            image_embeddings = image_embeddings / image_embeddings.norm(dim=-1, keepdim=True)

    return {
        "text_embedding": text_embeddings.numpy() if text_embeddings is not None else None,
        "image_embedding": image_embeddings.numpy() if image_embeddings is not None else None
    }


# Define the correct directories
text_dir = "../data/data/text"
table_dir = "../data/data/tables"
image_dir = "../data/data/images"

# Collect files
text_files = [os.path.join(text_dir, f) for f in os.listdir(text_dir) if f.endswith(".txt")]
table_files = [os.path.join(table_dir, f) for f in os.listdir(table_dir) if f.endswith(".txt")]
image_files = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(".png")]

# Prepare items
items = []
for text_file in text_files:
    with open(text_file, "r", encoding="utf-8") as f:
        text = f.read().strip()
        if text:
            items.append({"type": "text", "text": text})
for table_file in table_files:
    with open(table_file, "r", encoding="utf-8") as f:
        table = f.read().strip()
        if table:
            items.append({"type": "table", "text": table})
for image_file in image_files:
    if os.path.exists(image_file):
        items.append({"type": "image", "image": image_file})

# Generate embeddings
embedding_vector_dimension = 384
valid_embeddings = []

with tqdm(total=len(items), desc="Generating embeddings") as pbar:
    for item in items:
        try:
            if item["type"] in ["text", "table"]:
                embedding = generate_multimodal_embeddings(prompt=item["text"])["text_embedding"]
            elif item["type"] == "image":
                embedding = generate_multimodal_embeddings(image_path=item["image"])["image_embedding"]
            if embedding is not None:
                valid_embeddings.append(embedding)
        except Exception as e:
            print(f"Failed to process {item}: {e}")
        pbar.update(1)

# Build FAISS Index
if valid_embeddings:
    all_embeddings = np.array(valid_embeddings, dtype=np.float32)
    if len(all_embeddings.shape) == 1:
        all_embeddings = all_embeddings.reshape(1, -1)
    index = faiss.IndexFlatL2(embedding_vector_dimension)
    index.add(all_embeddings)
    print(f"FAISS index built with {index.ntotal} embeddings.")
else:
    print("No valid embeddings generated.")

Generating embeddings:   0%|                                                                                 | 0/89 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   1%|▊                                                                        | 1/89 [00:06<08:49,  6.02s/it]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   2%|█▋                                                                       | 2/89 [00:10<07:42,  5.31s/it]

...

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:  99%|███████████████████████████████████████████████████████████████████████▏| 88/89 [17:20<00:12, 12.41s/it]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings: 100%|████████████████████████████████████████████████████████████████████████| 89/89 [17:25<00:00, 11.75s/it]

No valid embeddings generated.
